In [1]:
import cv2
import mediapipe as mp
import rtmidi


['Microsoft GS Wavetable Synth 0', 'p10 1', 'p2 2', 'loopMIDI Port 3', 'loopMIDI Port 1 4']


SystemError: MidiOutWinMM::openPort: error creating Windows MM MIDI output port.

In [1]:
import cv2
import mediapipe as mp
import mido
import time

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Set up MIDI output
midi_port_name = 'loopMIDI Port 1 4'  
midi_out = mido.open_output(midi_port_name)

def count_fingers(hand_landmarks):
    finger_tips = [4, 8, 12, 16, 20]
    finger_count = 0

    if hand_landmarks:
        for tip in finger_tips:
            if tip == 4:  # Thumb
                if hand_landmarks.landmark[tip].x < hand_landmarks.landmark[tip - 1].x:
                    finger_count += 1
            elif hand_landmarks.landmark[tip].y < hand_landmarks.landmark[tip - 2].y:
                finger_count += 1

    return finger_count

def send_midi_note(finger_count):
    # Map finger count to MIDI note numbers
    note_mapping = {
        1: 60,  # C4
        2: 62,  # D4
        3: 64,  # E4
        4: 65,  # F4
        5: 67   # G4
    }
    
    if finger_count in note_mapping:
        note = note_mapping[finger_count]
        # Send note on
        midi_out.send(mido.Message('note_on', note=note, velocity=64))
        # Wait a short duration
        time.sleep(0.1)
        # Send note off
        midi_out.send(mido.Message('note_off', note=note, velocity=64))

last_sent_time = 0
cooldown = 0.5  

cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
                
                finger_count = count_fingers(hand_landmarks)
                cv2.putText(image, f'Fingers: {finger_count}', (10, 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                
                current_time = time.time()
                if current_time - last_sent_time > cooldown:
                    send_midi_note(finger_count)
                    last_sent_time = current_time

        image = cv2.flip(image, 1)
        cv2.imshow('Hands', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()
midi_out.close() 